In [ ]:
# importar librerías
import pandas as pd
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt

Primero importaremos las librerías necesarias para examinar las tablas; esto nos permitirá obtener una visión general de los datos.

In [ ]:
# cargar archivos
traffic = pd.read_csv('C:/Users/pasty/Documents/datasets/tomtom_traffic.csv')
eco = pd.read_csv('C:/Users/pasty/Documents/datasets/oecd_city_economy.csv')

In [ ]:
# mostrar las primeras 5 filas de traffic
traffic.head(5)

In [ ]:
# Examinar la estructura de traffic
traffic.info(3)

In [ ]:
# mostrar las primeras 5 filas de eco
eco.head(5)

In [ ]:

# Examinar la estructura de eco
eco.info(3)

Con el panorama general claro, procederemos a modificar los nombres de las columnas. El objetivo es facilitar la unión entre tablas y asignar nombres más intuitivos que mejoren la legibilidad de los datos.

In [ ]:
# Estandarizar los nombres de las columnas de traffic
traffic=traffic.rename(columns={'Country':'country', 'City':'city', 'UpdateTimeUTC':'Update_Time_UTC', 'JamsDelay':'Jams_Delay', 'TrafficIndexLive':'Traffic_Index_Live',
       'JamsLengthInKms':'Jams_Length_In_Kms', 'JamsCount':'Jams_Count', 'TrafficIndexWeekAgo':'Traffic_Index_Week_Ago',
       'UpdateTimeUTCWeekAgo':'Update_Time_UTC_Week_Ago', 'TravelTimeLivePer10kmsMins':'Travel_Time_Live_Per_10_kms_Mins',
       'TravelTimeHistoricPer10KmsMins':'Travel_Time_Historic_Per_10_Kms_Mins','MinsDelay': 'Mins_Delay'})

traffic.columns=traffic.columns.str.lower()

# verificar cambios
traffic.columns


In [ ]:
# Estandarizar los nombres de las columnas de eco
eco=eco.rename(columns={'Year':'year','City':'city','Country':'country','City GDP/capita':'city_gdp_capita','Population (M)':'population_m','PM2.5 (μg/m³)':'PM2.5(μg/m³)','Unemployment %':'unemployment_pct'})
# verificar cambios
eco.columns

Para continuar con la unión, debemos estandarizar los tipos de datos; es decir, corregir espacios, puntos y guiones, o convertir valores de texto a formato numérico o de fecha. Esto tiene como fin facilitar la integración y el trabajo posterior con el dataset.

In [ ]:
# Convertir las columnas de traffic a tipo fecha con pd.to_datetime()
traffic['update_time_utc'] =pd.to_datetime(traffic['update_time_utc'],errors='coerce',utc=True) 
traffic['update_time_utc_week_ago'] =pd.to_datetime(traffic['update_time_utc_week_ago'],errors='coerce',utc=True) 

traffic.info()

In [ ]:
# Limpia separadores y convierte columnas numéricas en eco
eco['city_gdp_capita'] =eco['city_gdp_capita'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)
eco['unemployment_pct'] =eco['unemployment_pct'].astype(str).str.replace('%', '').str.replace(',', '.').astype(float) 
eco['population_m'] =eco['population_m'].astype(str).str.replace(',', '.').astype(float)

# Calcula la población total en unidades absolutas (Multiplica * 1000000)
eco['population'] = eco['population_m']*1000000


# verificar el cambio
eco.info()
eco.head(3)

En este apartado, extraemos el año de las fechas para delimitar nuestro análisis. Para este proyecto, trabajaremos específicamente con los datos correspondientes al 2024

In [ ]:
# Extraer el año de las fechas en update_time_utc
traffic['year'] = traffic['update_time_utc'].dt.year

# Verificar cambio
traffic.head(3)

In [ ]:
# Filtra los registros del año 2024
traffic_2024 = traffic[traffic['year']==2024] .copy()
eco_2024 = eco[eco['year']==2024] .copy()

# Revisar dataframes nuevos
display(traffic_2024.head())
display(eco_2024.head())

También es importante definir qué mediciones y agrupaciones utilizaremos para facilitar tanto la lectura como la interpretación de los resultados.

In [ ]:
# Calcular los  promedios de trafico por ciudad, país y año
traffic_city_year_2024 = traffic_2024.groupby(['city', 'country', 'year'])[['jams_delay','traffic_index_live','jams_length_in_kms','jams_count','mins_delay','traveltimeliveper10kmsmins','travel_time_historic_per_10_kms_mins']].mean().reset_index() 
# Mostrar resultado
traffic_city_year_2024.head()

In [ ]:
#ordenamiento de datos por jams_delay descendente
traffic_city_year_2024.sort_values(["jams_delay"], ascending=False)

Llegamos al paso de la unión de tablas, donde definiremos qué columnas integrar y cuál será la clave en común. En todo momento, priorizamos que la tabla resultante esté ordenada y contenga únicamente los datos más relevantes para el análisis.

In [ ]:
# Seleccionar columnas clave de tráfico y economía
left_cols = ['year', 'city', 'jams_delay','traffic_index_live', 'jams_length_in_kms', 'jams_count','mins_delay']

right_cols = ['year', 'city', 'country', 'city_gdp_capita', 'unemployment_pct','PM2.5(μg/m³)', 'population']

# Usar .copy() para crear los dos nuevos datasets reducidos
traffic_2024_small = traffic_city_year_2024[left_cols].copy()
eco_2024_small = eco_2024[right_cols].copy()

# Unir datasets
merged = pd.merge(traffic_2024_small,eco_2024_small, on=['year', 'city'],how='inner') # tu código aquí

# Mostrar las primeras 5 filas
print(merged.head())
print(merged.columns.tolist())
# reordenar columnas
columnas_ordenadas=['year', 'city','country', 'jams_delay', 'traffic_index_live', 'mins_delay', 'city_gdp_capita', 'unemployment_pct', 'population']
print(merged['city'].unique().tolist())
print(merged['country'].unique().tolist())
merged=merged[columnas_ordenadas]
print(merged.head())

Generamos gráficos para visualizar la distribución de los datos, comenzando por el tráfico. Al observar el boxplot, notamos una gran dispersión en los retrasos (jams_delay) y la presencia de valores atípicos, aunque el promedio general se sitúa en 629.52.

In [ ]:
mean_value = merged['jams_delay'].mean()
sns.boxplot(data=merged,y='jams_delay')
plt.title(f'Boxplot de JamsDelay (2024)\nPromedio: {mean_value:.2f}')
plt.show()


En segundo lugar, analizaremos la distribución del ingreso per cápita por ciudad. A través de este histograma, observamos que la mayoría de las ciudades se concentran en un rango cercano al promedio de 13,253.60 USD, aunque existe una dispersión importante con casos que superan los 
20,000 USD y en el caso menor de 5,000 USD.

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(merged['city_gdp_capita'],color='skyblue', bins=15)
promedio_gdp = merged['city_gdp_capita'].mean()
plt.axvline(promedio_gdp, color='red', linestyle='--', linewidth=2, label=f'Promedio: {promedio_gdp:.2f}')
plt.title('Distribución del PIB per Cápita por Ciudad (2024)')
plt.xlabel('PIB per Cápita (USD)')
plt.ylabel('Frecuencia (Número de ciudades)')
plt.legend()
plt.show()


Por último, evaluamos la relación entre el tráfico y el ingreso per cápita. Aunque se podría pensar que un mayor ingreso implica mayor tráfico, los gráficos sugieren que existen otros factores determinantes. Se puede suponer que factores como el diseño urbanístico, la eficiencia del transporte público o la infraestructura vial parecen tener un impacto más significativo en la congestión que el nivel económico por sí solo.

In [ ]:
merged_ordenado = merged.sort_values(by='city_gdp_capita', ascending=True)
merged_ordenado.plot(x='city', y=['jams_delay', 'city_gdp_capita'], kind='bar', figsize=(10,6))
plt.title('Comparación: Atascos vs Economía por Ciudad')
plt.xticks(rotation=90)
plt.show()

merged_plot = merged_ordenado.copy()
merged_plot['traffic_index_scaled'] = merged_plot['traffic_index_live'] * 100
merged_plot.plot(x='city', y=['traffic_index_scaled', 'city_gdp_capita'], kind='bar', figsize=(10,6))
plt.title('Comparacion:Indice de Atascos vs Economia por Ciudad')
plt.ylabel('Valores (Traffic Index x100, GDP per capita)')
plt.xticks(rotation=90)
plt.legend(['Traffic Index (x100)', 'GDP per capita'])
plt.show()

In [ ]:
# Exporta el dataset final como CSV
merged.to_csv('movilidad_economica_final.csv', index=False, encoding='utf-8-sig')